In [1]:
import paho.mqtt.client as mqtt
import redis
import json

TOPIC = 's302294'
mqtt_client = mqtt.Client()
redis_client = redis.Redis(
  username='default',
  host='redis-19264.c301.ap-south-1-1.ec2.cloud.redislabs.com',
  port=19264,
  password='6HVuWRH8CSONqykRKS9Y2esqvv6o4II7')
  
def on_connect(client, userdata, flags, rc):
    client.subscribe(TOPIC)
        

def on_message(client, userdata, msg):
    battery_info = json.loads(msg.payload.decode())
    for attribute in ['battery', 'power']:
        ts_name = ':'.join((battery_info['mac_address'], attribute))
        if ts_name not in redis_client.keys():
            safe_create_ts(redis_client, ts_name)
        safe_add_ts(redis_client, ts_name, battery_info['timestamp'], battery_info['battery_level'] if attribute == 'battery' else int(battery_info['power_plugged']))


def safe_create_ts(client, ts_name):
      if client.ping():
        try:
           client.ts().create(key=ts_name)
        except redis.ResponseError:
            pass

def safe_add_ts(client, ts_name, timestamp, value):
      if client.ping():
        try:
           client.ts().add(key=ts_name, timestamp=timestamp, value=value)
        except redis.ResponseError:
            pass
    

mqtt_client.on_connect = on_connect
mqtt_client.on_message = on_message

mqtt_client.connect('mqtt.eclipseprojects.io', 1883)

# Start the client loop to process network events and call the callbacks
mqtt_client.loop_forever()

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6e19e91f-1616-4474-bfab-13605cd1d677' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>